# CSE 5243 - Introduction to Data Mining
## Homework 5: Association Analysis
- Semester: Fall 2024
- Instructor: Thomas Bihari
- Section: TuTh 12:45pm
- Student Name: Mary Shas Anak Rowin Umpok
- Student Email: anakrowinumpok.1@osu.edu
- Student ID: 500611954

Template Version V1.
***

# Introduction

### Objectives

In this lab, you will use the "TEB_Groceries_dataset3.xlsx" dataset provided on Carmen to find potential association rules.

The objectives of this assignment are:
- Practice the Association Analysis content we covered this semester.
- Understand “why” the particular topics, techniques, etc., are important from a practical perspective.
- Understand how to choose and use appropriate tools to solve the provided problems.

### The Dataset
- This workbook contains is a market basket dataset containing transactions.
- The data file captures the data in "long format". Specifically, every row corresponds to the transaction id and the item. If the specific transaction id has multiple items, there will be multiple rows in the data.
- You can process the data however you like, but it is recommended you convert into a one-hot-encoded data structure. This will allow you to easily use the mlxtend package.

## The Business Problem
- Assume this dataset contains all of the transactions for one month for our store.  We wish to find association rules that would improve our revenue as follows:
  - We would discount one of our products by 10% each month, with the hope that this would encourage customers to visit our store to purchase that product 5% more frequently, and also purchase  other products (that are not discounted) more frequently.
- Practically speaking, we would like to come up with **two-item** rules (one antecedent and one consequent: (A -> B)) and choose the one that best adds to our revenues (based on the rule support, confidence, etc.).

### Collaboration
For this assignment, you should work as an individual. You may informally discuss ideas with classmates, but your work should be your own.

### What You Need to Turn In
- Submit this Jupyter Notebook in .IPYNB format.  Do not "zip" the file.

### Notes
- Feel free to use the **mlxtend** package throughout this assignment.
- If a question asks you to "calculate" the number of "all possible rules", etc., explain the calculation by showing the "formula" you used. This will act as "showing your work".
***

***
# Section: 1 - Get Ready
1A) Load the data, and get it ready for association analysis. Do this with convenient python helper methods as appropriate. Feel free to use the tools given in the example we covered. 
- Suggest: Make the data one-hot encoded.
***

In [108]:
#Note: If the mlxtend library is not installed, uncomment the following line (once) and run it.
#!pip install mlxtend
import decimal
import numpy as np
import pandas as pd
import mlxtend as mlx
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

pd.set_option('display.max_columns', 1000) #include to avoid ... in middle of display (and use 'display(...)' when printing in cells)
pd.set_option('display.max_rows', 1000) #include to avoid ... in middle of display (and use 'display(...)' when printing in cells) COMMENT OUT TO SHORTEN OUTPUTS

In [109]:
price_discount_factor = 0.90   #adjustable
sales_increase_factor = 1.05   #adjustable

In [110]:
# load the Item data
items_df = pd.read_excel('TEB_Groceries_dataset3.xlsx', sheet_name='Items')
display(items_df.info())
display(items_df.describe(include="all").T)
items_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ItemID     167 non-null    int64  
 1   Name       167 non-null    object 
 2   UnitPrice  167 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.0+ KB


None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ItemID,167.0,NaN,NaN,NaN,84.0,48.35287,1.0,42.5,84.0,125.5,167.0
Name,167,167,abrasive cleaner,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,167.0,NaN,NaN,NaN,5.040299,2.813436,0.07,2.62,4.96,7.535,9.91


,ItemID,Name,UnitPrice
0,1,abrasive cleaner,1.23
1,2,artif. sweetener,8.24
2,3,baby cosmetics,7.56
3,4,bags,8.75
4,5,baking powder,6.43


In [111]:
# load the Transaction data
trans_df = pd.read_excel('TEB_Groceries_dataset3.xlsx', sheet_name='TxToItem')
display(trans_df.info())
display(trans_df.describe(include="all").T)
trans_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TxID    38765 non-null  int64
 1   ItemID  38765 non-null  int64
dtypes: int64(2)
memory usage: 605.8 KB


None

,count,mean,std,min,25%,50%,75%,max
TxID,38765.0,7470.155553,4322.026662,1.0,3733.0,7466.0,11195.0,14963.0
ItemID,38765.0,92.829073,52.260057,1.0,43.0,102.0,138.0,167.0


,TxID,ItemID
0,1,165
1,1,105
2,1,128
3,2,130
4,2,165


In [112]:
# One hot encode the Transaction data
# orders_df is a "long form" dataset. We first convert it to "wide form" then one-hot encode it.
from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict

transaction_items = defaultdict(list)
for transaction in trans_df.values.tolist():
    transaction_items[transaction[0]].append(transaction[1])
    
dataset_wide = list(transaction_items.values())

te = TransactionEncoder()
te_ary = te.fit(dataset_wide).transform(dataset_wide)

ohe_df= pd.DataFrame(te_ary, columns=te.columns_)
display(ohe_df.head(5))

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

***
# Section: 2 - Explore the Data
***

***
## Section: 2.1 - Get the Order and Product Sizes
- Calculate the **number_of_orders** and **number_of_products**.
***

In [113]:
transactions_df = pd.read_excel('TEB_Groceries_dataset3.xlsx', sheet_name='Transactions')
number_of_orders = transactions_df['TxID'].count()
print("Number of orders: ", number_of_orders)

items_df = pd.read_excel('TEB_Groceries_dataset3.xlsx', sheet_name='Items')
number_of_products = items_df['ItemID'].count()
print("Number of products: ", number_of_products)

Number of orders:  14963
Number of products:  167


***
## Section: 2.2 - Evaluate the Itemset and Rule Size & Complexity
- Calculate the **maximum number of Itemsets** that could be created from the items (without considering the actual transaction data). Show your work.
- Calculate the **maximum number of Rules** that can be created from the items (without considering the actual transaction data). Show your work.
- What do the calculations suggest as a **potential cause of concern**? Hint: Complexity.
- What might you do to manage these concerns?
***

In [114]:
from math import pow

max_num_itemsets = pow(2, number_of_products)
print("Maximum number of itemsets: ", max_num_itemsets)

max_num_rules = pow(2, number_of_products) - number_of_products - 1
print("Maximum number of rules: ", max_num_rules)

Maximum number of itemsets:  1.8707220957835557e+50
Maximum number of rules:  1.8707220957835557e+50


**Potential cause of concern:**
The values above suggest that the size of the itemsets are very big and the complexity of the problem increases exponentially. This might cause a problem when it comes to computing the rules and it's going to take up a huge amount of memory.

**Managing the concern:**
We can use support, confidence, or lift thresholds to focus on frequent and meaningful patterns, which helps in reducing the number of itemsets and rules.

***
# Section: 3 - Itemset Generation
***

***
## Section: 3.1 - Revise the Dataset
- If/as appropriate, trim or revise the dataset to make the runtime reasonable.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [115]:
# May not be needed.  See how things work below, and then trim the datasets, etc., if needed. (Ask for guidance first, though.)

***
## Section: 3.2 - Create Two-Itemsets
- Create a set of about 50 to 100 two-item sets with highest support. Sort them in decreasing order of support.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [116]:
frequent_itemsets_ap = mlx.frequent_patterns.apriori(ohe_df, min_support=0.0040, use_colnames=True)

two_itemsets = frequent_itemsets_ap[frequent_itemsets_ap['itemsets'].apply(lambda x: len(x) == 2)]

sorted_two_itemsets = two_itemsets.sort_values(by='support', ascending=False)

sorted_two_itemsets = sorted_two_itemsets.reset_index(drop=True)

sorted_two_itemsets

,support,itemsets
0,0.014837,"(165, 102)"
1,0.013968,"(122, 165)"
2,0.011629,"(138, 165)"
3,0.011161,"(165, 166)"
4,0.010559,"(122, 102)"
5,0.009691,"(138, 102)"
6,0.008955,"(130, 165)"
7,0.008220,"(156, 165)"
8,0.008087,"(122, 138)"
9,0.008087,"(166, 102)"


**Discussion:**
I used the Apriori algorithm to create the frequent itemsets. After creating the frequent itemsets, I filtered it so it only contains two itemsets, and I sort the results in decreasing order of support. Before printing out the result, I reset the index so it clearly show the amount of itemsets.

Based on the result, when I set the support to 0.0040, we get a total of 61 two itemsets. The two itemset with the highest support of 0.014837 is itemset (165, 102).

***
# Section: 4 - Generate Rules
- For the two-itemsets created above, create the related rules.
***

In [117]:
rules_ap = mlx.frequent_patterns.association_rules(frequent_itemsets_ap, metric="confidence", min_threshold=0.13, num_itemsets=frequent_itemsets_ap.shape[0])
sorted_rules_ap = rules_ap.reset_index(drop=True)
sorted_rules_ap

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(7),(165),0.033950,0.157923,0.004678,0.137795,0.872548,1.0,-0.000683,0.976656,-0.131343,0.024991,-0.023902,0.083709
1,(10),(165),0.045312,0.157923,0.007151,0.157817,0.999330,1.0,-0.000005,0.999874,-0.000702,0.036469,-0.000126,0.101549
2,(14),(165),0.035220,0.157923,0.004678,0.132827,0.841090,1.0,-0.000884,0.971060,-0.163761,0.024823,-0.029802,0.081225
3,(29),(165),0.053131,0.157923,0.007151,0.134591,0.852259,1.0,-0.001240,0.973040,-0.154748,0.035070,-0.027707,0.089936
4,(48),(165),0.037091,0.157923,0.005280,0.142342,0.901341,1.0,-0.000578,0.981834,-0.102072,0.027827,-0.018503,0.087887
5,(55),(102),0.037760,0.122101,0.005146,0.136283,1.116150,1.0,0.000536,1.016420,0.108146,0.033261,0.016154,0.089214
6,(55),(165),0.037760,0.157923,0.005280,0.139823,0.885388,1.0,-0.000683,0.978958,-0.118576,0.027729,-0.021494,0.086628
7,(95),(165),0.038896,0.157923,0.005614,0.144330,0.913926,1.0,-0.000529,0.984114,-0.089246,0.029360,-0.016142,0.089939
8,(109),(165),0.049054,0.157923,0.006616,0.134877,0.854071,1.0,-0.001130,0.973362,-0.152310,0.033022,-0.027367,0.088387
9,(111),(165),0.037091,0.157923,0.005012,0.135135,0.855703,1.0,-0.000845,0.973652,-0.149027,0.026381,-0.027061,0.083437


***
# Section: 5 - Rule Evaluation
- For the rules created above, find the product (that would be given the discount) that would cause the greatest increase in monthly store revenue.
  - This is based on the Business Problem stated at the top of this notebook.
  - Explain your choice and give supporting evidence.
  - Consider:
    - How much will the store's monthly revenue decrease (or increase) due to the change in price for the chosen Products (and its increased sales)?
    - How much will the store's monthly revenue increase (or decrease) due to the increased sales of the associated Products?
  
***

In [118]:
for index, row in rules_ap.iterrows():
    ant = list(rules_ap['antecedents'][index])
    cons = list(rules_ap['consequents'][index])
    conf = rules_ap['confidence'][index]
    support = rules_ap['support'][index]

    print("ANTECEDENT: ", ant)

    ant_product_name = items_df[items_df['ItemID'].isin(ant)]['Name'].values
    print("Product name: ", ant_product_name)
          
    selected_product_price = items_df[items_df['ItemID'].isin(ant)]['UnitPrice'].values
    discounted_product_price = price_discount_factor * selected_product_price

    ant_transaction_count = len(trans_df[trans_df['ItemID'].isin(ant)])
    print("Old sales count: ", ant_transaction_count)
    
    updated_transaction_count = ant_transaction_count * sales_increase_factor
    print("New sales count: ", math.floor(updated_transaction_count))
    
    ant_old_revenue = selected_product_price * ant_transaction_count

    ant_new_revenue = discounted_product_price * updated_transaction_count

    ant_change_in_revenue = ant_new_revenue - ant_old_revenue
    print("Change in revenue: ", np.round(ant_change_in_revenue, 2))

    print(" ")
    print("CONSEQUENT: ", cons)

    cons_product_name = items_df[items_df['ItemID'].isin(cons)]['Name'].values
    print("Product name: ", cons_product_name)

    additional_trans_cons = conf * updated_transaction_count
    print("Number of additional transaction for consequence ", cons, ": ", math.floor(additional_trans_cons))

    cons_price = items_df[items_df['ItemID'].isin(ant)]['UnitPrice'].values

    cons_transaction_count = len(trans_df[trans_df['ItemID'].isin(cons)]) 
    print("Old sales count: ", cons_transaction_count)  

    cons_old_revenue = cons_price * cons_transaction_count

    cons_count = support * number_of_orders

    cons_new_revenue = cons_price * (additional_trans_cons + cons_transaction_count - cons_count)

    cons_change_in_revenue = cons_new_revenue - cons_old_revenue
    print("Change in revenue: ", np.round(cons_change_in_revenue, 2))

    print(" ")

    total_old_revenue = ant_old_revenue + cons_new_revenue
    print("TOTAL OLD REVENUE: ", np.round(total_old_revenue, 2))

    total_new_revenue = ant_new_revenue + cons_new_revenue
    print("TOTAL NEW REVENUE: ", np.round(total_new_revenue, 2))

    change_in_total_revenue = total_new_revenue - total_old_revenue
    print("CHANGE IN TOTAL REVENUE: ", np.round(change_in_total_revenue, 2))

    print(" ")
    print("******************************************************")
    print(" ")

ANTECEDENT:  [7]
Product name:  ['beef']
Old sales count:  516
New sales count:  541
Change in revenue:  [-59.88]
 
CONSEQUENT:  [165]
Product name:  ['whole milk']
Number of additional transaction for consequence  [165] :  74
Old sales count:  2502
Change in revenue:  [9.83]
 
TOTAL OLD REVENUE:  [6377.81]
TOTAL NEW REVENUE:  [6317.93]
CHANGE IN TOTAL REVENUE:  [-59.88]
 
******************************************************
 
ANTECEDENT:  [10]
Product name:  ['bottled beer']
Old sales count:  687
New sales count:  721
Change in revenue:  [-292.83]
 
CONSEQUENT:  [165]
Product name:  ['whole milk']
Number of additional transaction for consequence  [165] :  113
Old sales count:  2502
Change in revenue:  [53.02]
 
TOTAL OLD REVENUE:  [24767.77]
TOTAL NEW REVENUE:  [24474.94]
CHANGE IN TOTAL REVENUE:  [-292.83]
 
******************************************************
 
ANTECEDENT:  [14]
Product name:  ['butter']
Old sales count:  534
New sales count:  560
Change in revenue:  [-127.47]
 

**Discussion:**
Based on my calculation above for all the rules, none of the rules produce a positive change in revenue. However, if we were to choose one product that would get the discount, it would be the beef with whole milk as the consequent because it produce the smallest negative change in total revenue.


***
# Section: 6 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

**Conclusions:**
This homework has taught me what does antecent support, consequent support, support, and confidence means and how it can help us to make informed decision in our business. Support means how frequent the itemsets appear in our transaction and confidence means how sure are that the customer will buy item B given that the customer buy item A. Given the min_threshold, there are in total 11 rules generated, and based on these 12 rules, we can predict that the itemsets [beef, whole milk] would cause the greatest increase in the total monthly store revenue. Therefore, association analysis is a great tool for business entities to use to predict their future sales.

***
### END-OF-SUBMISSION
***